In [3]:
import TensorFlow

In [5]:
public struct aModel: Layer {
    public var conv = Conv2D<Float>(
        filterShape: (3, 3, 16, 32), 
        strides: (2, 2), 
        padding: .same, 
        activation: relu)
    public var pool = GlobalAvgPool2D<Float>()
    public var linear = Dense<Float>(inputSize: 32, outputSize: 10)
    
    @differentiable
    public func call(_ input: Tensor<Float>) -> Tensor<Float> {
        return input.sequenced(through: conv, pool, linear)
    }
}

In [6]:
var model = aModel()

In [7]:
let x = Tensor<Float>(randomNormal: [64, 10, 10, 16])

In [8]:
let pred = model(x)
pred.shape

▿ TensorShape
  ▿ dimensions : 2 elements
    - 0 : 64
    - 1 : 10


In [9]:
let y = Tensor<Float>(randomNormal: [64, 10])

In [13]:
let (loss, grad) = model.valueWithGradient {model -> Tensor<Float> in
    return meanSquaredError(predicted: model(x), expected: y)}

Fatal error: Incompatible shapes: [64,32] vs. [64,5,5,32]: file /swift-base/swift/stdlib/public/TensorFlow/CompilerRuntime.swift, line 2108
Current stack trace:
0    libswiftCore.so                    0x00007fe34cfc9e00 _swift_stdlib_reportFatalErrorInFile + 115
1    libswiftCore.so                    0x00007fe34cf1206c <unavailable> + 3035244
2    libswiftCore.so                    0x00007fe34cf1215e <unavailable> + 3035486
3    libswiftCore.so                    0x00007fe34cd59a12 <unavailable> + 1231378
4    libswiftCore.so                    0x00007fe34ceded42 <unavailable> + 2825538
5    libswiftCore.so                    0x00007fe34cd58ef9 <unavailable> + 1228537
6    libswiftTensorFlow.so              0x00007fe34a349162 <unavailable> + 598370
7    libswiftTensorFlow.so              0x00007fe34a3478b0 checkOk(_:file:line:) + 508
8    libswiftTensorFlow.so              0x00007fe34a36bfb0 _TFCCheckOk(_:) + 81
9    libswiftTensorFlow.so              0x00007fe34a36bfa0 _swift_tfc_Che

: 

In [15]:
let y = Tensor<Float>(randomNormal: [64, 16])

In [18]:
let z = x.mean(squeezingAxes: [1, 2])
(y-z).mean()

0.02765454


In [17]:
let (loss, grad) = valueWithGradient(at: x) { x -> Tensor<Float> in
    let z = x.mean(squeezingAxes: [1, 2])
    return (y-z).mean()}

Fatal error: Incompatible shapes: [64,16] vs. [64,10,10,16]: file /swift-base/swift/stdlib/public/TensorFlow/CompilerRuntime.swift, line 2108
Current stack trace:
0    libswiftCore.so                    0x00007fe34cfc9e00 _swift_stdlib_reportFatalErrorInFile + 115
1    libswiftCore.so                    0x00007fe34cf1206c <unavailable> + 3035244
2    libswiftCore.so                    0x00007fe34cf1215e <unavailable> + 3035486
3    libswiftCore.so                    0x00007fe34cd59a12 <unavailable> + 1231378
4    libswiftCore.so                    0x00007fe34ceded42 <unavailable> + 2825538
5    libswiftCore.so                    0x00007fe34cd58ef9 <unavailable> + 1228537
6    libswiftTensorFlow.so              0x00007fe34a349162 <unavailable> + 598370
7    libswiftTensorFlow.so              0x00007fe34a3478b0 checkOk(_:file:line:) + 508
8    libswiftTensorFlow.so              0x00007fe34a36bfb0 _TFCCheckOk(_:) + 81
9    libswiftTensorFlow.so              0x00007fe34a36bfa0 _swift_tfc_C

: 